In [1]:
%cd /home/ducpham/Documents/LightRAG
# move to project

/home/ducpham/Documents/LightRAG


In [ ]:
# init global config for classes 

from config import ConfigParams
from dataclasses import asdict

config = asdict(ConfigParams())
for k, v in config.items():
    print(f"Key: {k}  -> Value: {v}")

#### 1. Init instance 

In [17]:
from lightrag.kvstore import JsonKVStorage
from tqdm.asyncio import tqdm as tqdm_async
from lightrag.utils import compute_mdhash_id, chunking_by_token_size


json_kv_full_docs = JsonKVStorage(
    namespace="json_cache_full_docs",
    global_config=config,
)


json_kv_chunks = JsonKVStorage(
    namespace="json_cache_chunks",
    global_config=config,
)

INFO:lightrag:Load KV json_cache_full_docs with 0 data
INFO:lightrag:Load KV json_cache_chunks with 0 data


#### 2. Make some data

In [20]:
"""Make full docs"""

with open("./testing/data/data.txt", mode='r') as f:
    data = [f.read()]

new_docs = {
    compute_mdhash_id(content=text.strip(), prefix="doc-"): {"content": text.strip()}
    for text in data
} 

print(new_docs)

{'doc-b6c60a6063de3a72adf173d3080a87ca': {'content': 'Attack on Titan (Shingeki no Kyojin) là một bộ manga và anime nổi tiếng kể về cuộc chiến sinh tồn của loài người trong một thế giới bị bao vây bởi những Titan khổng lồ. Câu chuyện tập trung vào Eren Yeager, Mikasa Ackerman và Armin Arlert - ba người bạn thân thiết sống trong những bức tường bảo vệ. Sau khi mẹ của Eren bị Titan ăn thịt trong cuộc tấn công vào Tường Maria, họ quyết định gia nhập Binh đoàn Trinh sát để chiến đấu chống lại những Titan. Trong hành trình của mình, họ dần khám phá ra nhiều bí mật động trời về nguồn gốc của Titan, sự thật về thế giới bên ngoài những bức tường, và vai trò của những nhân vật quan trọng như Zeke Yeager, Reiner Braun, Annie Leonhart và Levi Ackerman. Đặc biệt, việc Eren phát hiện ra khả năng biến thành Titan của mình đã mở ra một bước ngoặt quan trọng trong cốt truyện, dẫn đến những xung đột phức tạp về ý thức hệ và đạo đức giữa các phe phái, đồng thời đặt ra những câu hỏi sâu sắc về tự do, côn

In [21]:
"""Make chunks data"""

inserting_chunks = {}
for doc_key, doc in tqdm_async(
    iterable=new_docs.items(), 
    desc="Chunking docs", 
    unit="doc"):
    chunks = {
        compute_mdhash_id(content=dp['content'], prefix="chunk-"): {
            **dp, "full_doc_id": doc_key
        }
        for dp in chunking_by_token_size(
            content=doc['content'],
            max_token_size=20,
            overlap_token_size=5,
            tiktoken_model_name=config['tiktoken_model_name']
        )
    }
    inserting_chunks.update(chunks)

print(inserting_chunks)

Chunking docs: 100%|██████████| 1/1 [00:00<00:00, 2081.54doc/s]

{'chunk-9e0ddc96173a640e169206d171315211': {'tokens': 20, 'content': 'Attack on Titan (Shingeki no Kyojin) là một bộ manga và anime nổi tiếng', 'chunk_order_index': 0, 'full_doc_id': 'doc-b6c60a6063de3a72adf173d3080a87ca'}, 'chunk-46c1cc0fe51c7e669da21146c0d04362': {'tokens': 20, 'content': 'manga và anime nổi tiếng kể về cuộc chiến sinh tồn của loài người trong một thế giới bị', 'chunk_order_index': 1, 'full_doc_id': 'doc-b6c60a6063de3a72adf173d3080a87ca'}, 'chunk-4aac34dd3202888cee9f909f4d865789': {'tokens': 20, 'content': 'trong một thế giới bị bao vây bởi những Titan khổng lồ. Câu chuyện tập', 'chunk_order_index': 2, 'full_doc_id': 'doc-b6c60a6063de3a72adf173d3080a87ca'}, 'chunk-5b5243a30665b4227974ff4d6e232389': {'tokens': 20, 'content': '. Câu chuyện tập trung vào Eren Yeager, Mikasa Ackerman và Armin Ar', 'chunk_order_index': 3, 'full_doc_id': 'doc-b6c60a6063de3a72adf173d3080a87ca'}, 'chunk-6d53f05e5d034df723b7cef3e59baa3a': {'tokens': 20, 'content': 'erman và Armin Arlert - ba 

#### 3. Upsert into json_KV

In [18]:
await json_kv_full_docs.upsert(data=new_docs)
await json_kv_full_docs.index_done_callback()

In [19]:
await json_kv_chunks.upsert(data=inserting_chunks)
await json_kv_chunks.index_done_callback()